<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">TP : Aspirateur Robot</h1>

Une compagnie construit des robots aspirateurs qui se déplacent sur le sol et qui nettoient la surface sur laquelle ils passent.

Nous allons mettre en place une simulation pour étudier combien de temps met un groupe de robots pour nettoyer le sol d'une pièce en utilisant deux stratégies différentes.

Voici un modèle simplifié pour un unique robot se déplaçant dans une pièce carré $ 5 \times 5$ pour se faire une idée du fonctionnement de la simulation.

Le robot commence avec une position aléatoire dans la pièce et une direction de déplacement aléatoire.  
Les figures ci-dessous décrivent les positions du robot (représentés par un point noir) ainsi que les directions (représentées par une flèche rouge).

* temps $t=0$  


<div style="text-align: center">
<img src="Images/screen1.png" alt="Mouvement">
</div>

Le robot commence à la position $(2.1, 2.2)$ avec un angle de 205 degrés (dans le sens des aiguilles d'une montre depuis le Nord). Le carreau sur lequel il est est nettoyé.

* temps $t=1$  

<div style="text-align: center">
<img src="Images/screen2.png" alt="Mouvement">
</div>

Le robot s'est déplacé d'une unité dans la direction définie précédemment. Il arrive à la position $(1.7,1.3)$, nettoyant ainsi un nouveau carreau.

* temps $t=2$  


<div style="text-align: center">
<img src="Images/screen3.png" alt="Mouvement">
</div>

Le robot s'est déplacé dans la même direction (205° depuis le Nord) à la position $(1.2,0.4)$, nettoyant un nouveau carreau.

* temps $t=3$  

<div style="text-align: center">
<img src="Images/screen4.png" alt="Mouvement">
</div>

Le robot ne peut pas se déplacer d'une unité dans la même direction (sans taper le mur). En conséquent il choisit, aléatoirement, une nouvelle direction (ici 287°).

* temps $t=4$  

<div style="text-align: center">
<img src="Images/screen5.png" alt="Mouvement">
</div>

Le robot se déplace, selon sa nouvelle direction à la position $(0.3, 0.7)$, nettoyant un nouveau carreau.


## Détails de la simulation
Voici certaines informations sur la modélisation faite pour la simulation mise en place.
* Plusieurs robots (`Robot`)  
En général, il y a $N>0$ robots dans la pièce. Pour simplifier, on considère que les robots sont des points et qu'ils peuvent passer les uns sur les autres ou occuper la même position sans interférence.
* La pièce (`room`)  
La pièce est rectangulaire avec un nombre entier $w$ pour la largeur et $h$ pour la hauteur. Au départ le sol de la pièce est sale. Un robot ne peut pas aller au delà d'un mur de la pièce (un robot ne peut pas se déplacer vers un point qui est en dehors de la pièce).
* Les carreaux (`tile`)  
Il faut garder une trace des parties de la pièce qui ont été nettoyées par le(s) robot(s). La pièce est donc diviser en carreaus de dimension $1 \times 1$ (il y aura donc $w \times h$ carreaux dans la pièce). Lorsque le robot se situe sur n'importe quel point d'un carreau, on considère que le carreau est entièrement nettoyé. Par convention, on se réfère aux carreaux en utilisant un couple d'entiers 
$$(0,0), (0,1), ..., (0,h-1),(1,1), ..., (w-1, h-1)$$
* Règles de déplacements d'un robot  
    * Chaque robot a une position dans la pièce. Sa position est donnée par un couple de coordonnées $(x,y)$ qui sont des flottants vérifiant $0≤x<w$ et $0≤y<h$. Dans le programme, on utilisera des instances de la classe `Position` pour stocker ces coordonnées.
    * Un robot possède une direction pour son déplacement. Elle sera représentée par un entier $d$ vérifiant $0≤d<360$, qui correspond à un angle en degrés.
    * Tous les robots se daplacent à la même vitesse $s$, un flottant, qui est donné et reste constant au cours de la simulation. A chaque unité de temps, le robot se déplace, selon la direction de son déplacement, de $s$ unités.
    * Si un robot détecte qu'il va heurter le mur durant l'unité de temps, il ne se déplace pas mais choisit une nouvelle direction au hasard. Le robot tentera de se déplacer dans cette direction lors de la prochaine unité de temps, jusqu'à atteindre un nouveau mur.  

* Terminaison
La simulation termine lorsqu'une certaine proportion de carreaux de la pièce ont été nettoyés.



## Gestion de la pièce
Il faut commencer par implémenter deux classes pour garder une trace des parties de la pièces qui ont été nettoyés ainsi que la position et direction de chaque robot.

### Classe `Position`
Voici donc le début du programme avec la définition de la classe `Position` qui conserve les coordonnées $x$ et $y$ d'un robot dans la salle.

In [ ]:
import math
import random


class Position(object):
    """
    A Position represents a location in a two-dimensional room.
    """
    def __init__(self, x, y):
        """
        Initializes a position with coordinates (x, y).
        """
        self.x = x
        self.y = y
        
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def getNewPosition(self, angle, speed):
        """
        Computes and returns the new Position after a single clock-tick has
        passed, with this object as the current position, and with the
        specified angle and speed.

        Does NOT test whether the returned position fits inside the room.

        angle: number representing angle in degrees, 0 <= angle < 360
        speed: positive float representing speed

        Returns: a Position object representing the new position.
        """
        old_x, old_y = self.getX(), self.getY()
        angle = float(angle)
        # Compute the change in position
        delta_y = speed * math.cos(math.radians(angle))
        delta_x = speed * math.sin(math.radians(angle))
        # Add that to the existing position
        new_x = old_x + delta_x
        new_y = old_y + delta_y
        return Position(new_x, new_y)

    def __str__(self):  
        return "(%0.2f, %0.2f)" % (self.x, self.y)

### Classe `RectangularRoom`
Cette classe représente l'espace qui doit être nettoyé et garde une trace des carreaux qui ont déjà été nettoyés. 

Il faut implémenter la classe `RectangularRoom` en respectant les spécifications des docstrings.

Il faut donc pouvoir :
* Initialiser l'objet
* Signaler le carreau comme nettoyé lorsqu'un robot se déplace sur une position
* Déterminer si un carreau a été nettoyé
* Déterminer combien de carreaux il y a dans la pièce
* Déterminer combien de carreaux nettoyés il y a dans la pièce
* Générer une position aléatoire dans la pièce
* Déterminer si une position donnée est dans la pièce


In [ ]:
class RectangularRoom(object):
    """
    A RectangularRoom represents a rectangular region containing clean or dirty
    tiles.

    A room has a width and a height and contains (width * height) tiles. At any
    particular time, each of these tiles is either clean or dirty.
    """
    def __init__(self, width, height):
        """
        Initializes a rectangular room with the specified width and height.

        Initially, no tiles in the room have been cleaned.

        width: an integer > 0
        height: an integer > 0
        """
        raise NotImplementedError
    
    def cleanTileAtPosition(self, pos):
        """
        Mark the tile under the position POS as cleaned.

        Assumes that POS represents a valid position inside this room.

        pos: a Position
        """
        raise NotImplementedError

    def isTileCleaned(self, m, n):
        """
        Return True if the tile (m, n) has been cleaned.

        Assumes that (m, n) represents a valid tile inside the room.

        m: an integer
        n: an integer
        returns: True if (m, n) is cleaned, False otherwise
        """
        raise NotImplementedError
    
    def getNumTiles(self):
        """
        Return the total number of tiles in the room.

        returns: an integer
        """
        raise NotImplementedError

    def getNumCleanedTiles(self):
        """
        Return the total number of clean tiles in the room.

        returns: an integer
        """
        raise NotImplementedError

    def getRandomPosition(self):
        """
        Return a random position inside the room.

        returns: a Position object.
        """
        raise NotImplementedError

    def isPositionInRoom(self, pos):
        """
        Return True if pos is inside the room.

        pos: a Position object.
        returns: True if pos is in the room, False otherwise.
        """
        raise NotImplementedError

In [ ]:
# Test 1 : creation d'un objet
room = RectangularRoom(5, 5)
assert room.getNumTiles() == 25

In [ ]:
# Test 2 : test getNumTiles
room = RectangularRoom(3, 4)
assert room.getNumTiles() == 12
room = RectangularRoom(8, 3)
assert room.getNumTiles() == 24
room = RectangularRoom(2, 6)
assert room.getNumTiles() == 12
room = RectangularRoom(14, 7)
assert room.getNumTiles() == 98
room = RectangularRoom(14, 9)
assert room.getNumTiles() == 126

In [ ]:
# Test 3 : carreaux sales
room = RectangularRoom(4, 6)
assert room.getNumTiles() == 24

assert room.isTileCleaned(1, 2) == False
assert room.isTileCleaned(0, 0) == False
assert room.isTileCleaned(3, 5) == False

assert room.getNumCleanedTiles() == 0

In [ ]:
# Test 4 : carreaux nettoyés
room = RectangularRoom(3, 4)
assert room.getNumTiles() == 12
assert room.getNumCleanedTiles() == 0

for i in range(3):
    for j in range(4):
        position = Position(i, j)
        room.cleanTileAtPosition(position)
assert room.getNumCleanedTiles() == 12

In [ ]:
# Test 5 : carreaux nettoyés
room = RectangularRoom(6, 8)
assert room.getNumTiles() == 48
assert room.getNumCleanedTiles() == 0

pos1 = Position(3.2, 5.7)
room.cleanTileAtPosition(pos1)
assert room.isTileCleaned(3, 5) == True
pos2 = Position(5.1, 7.9)
room.cleanTileAtPosition(pos2)
assert room.isTileCleaned(5, 7) == True

assert room.getNumCleanedTiles() == 2

In [ ]:
# Test 6 : carreaux nettoyés plusieurs fois
room = RectangularRoom(12, 13)
for _ in range(5):
    for i in range(6):
        for j in range(4):
            position = Position(i, j)
            room.cleanTileAtPosition(position)
    assert room.getNumCleanedTiles() == 24

In [ ]:
# Test 7 : position aléatoire
room = RectangularRoom(6, 9)
for _ in range(100):
    position = room.getRandomPosition()
    assert 0 <= position.getX() < 6
    assert 0 <= position.getY() < 9

In [ ]:
# Test 8 : position dans la pièce
room = RectangularRoom(6, 9)
p1 = Position(0, 0)
assert room.isPositionInRoom(p1) == True
p2 = Position(5.9, 8.9)
assert room.isPositionInRoom(p2) == True
p3 = Position(6, 9)
assert room.isPositionInRoom(p3) == False
p4 = Position(8, 5)
assert room.isPositionInRoom(p4) == False
p5 = Position(5, 8)
assert room.isPositionInRoom(p5) == True
p6 = Position(-2, 8)
assert room.isPositionInRoom(p6) == False
p7 = Position(-2, -8)
assert room.isPositionInRoom(p7) == False
p8 = Position(2, -8)
assert room.isPositionInRoom(p8) == False
p9 = Position(12, 18)
assert room.isPositionInRoom(p9) == False

## La classe `Robot`
La classe `Robot` conserve la position et le direction du robot.  

Il faut implémenter la classe `Robot` en respectant les spécifications des docstrings.

Il faut donc pouvoir :
* Initialiser l'objet
* Accéder à la position du robot
* Accéder à la direction du robot
* Modifier la position du robot
* Modifier la direction du robot

### Remarques
* Lorsque le robot est initialisé, il doit nettoyer le carreau sur lequel il est créé. Le robot nettoie la case après être arrivé sur une case et l'on considère que la case entière est nettoyer (il suffit de prendre des carreaux de petites tailles pour que cette approximation soit bonne).
* La classe `Robot` est une **classe abstraite** : elle sert de base pour d'autres classes mais ne sera pas utilisée pour créer des instances.

In [ ]:
class Robot(object):
    """
    Represents a robot cleaning a particular room.

    At all times the robot has a particular position and direction in the room.
    The robot also has a fixed speed.

    Subclasses of Robot should provide movement strategies by implementing
    updatePositionAndClean(), which simulates a single time-step.
    """
    def __init__(self, room, speed):
        """
        Initializes a Robot with the given speed in the specified room. The
        robot initially has a random direction and a random position in the
        room. The robot cleans the tile it is on.

        room:  a RectangularRoom object.
        speed: a float (speed > 0)
        """
        raise NotImplementedError

    def getRobotPosition(self):
        """
        Return the position of the robot.

        returns: a Position object giving the robot's position.
        """
        raise NotImplementedError
    
    def getRobotDirection(self):
        """
        Return the direction of the robot.

        returns: an integer d giving the direction of the robot as an angle in
        degrees, 0 <= d < 360.
        """
        raise NotImplementedError

    def setRobotPosition(self, position):
        """
        Set the position of the robot to POSITION.

        position: a Position object.
        """
        raise NotImplementedError

    def setRobotDirection(self, direction):
        """
        Set the direction of the robot to DIRECTION.

        direction: integer representing an angle in degrees
        """
        raise NotImplementedError

    def updatePositionAndClean(self):
        """
        Simulate the passage of a single time-step.

        Move the robot to a new position and mark the tile it is on as having
        been cleaned.
        """
        raise NotImplementedError # don't change this!



In [ ]:
# Test 1 : créer un robot (même si nous ne le ferons plus)
room = RectangularRoom(1,2)
speed = 1.0
robot = Robot(room, speed)

In [ ]:
# Test 2 : tester la position
robot = Robot(RectangularRoom(5, 8), 1.0)
pos = robot.getRobotPosition()
assert 0 <= pos.getX() < 5
assert 0 <= pos.getY() < 8

In [ ]:
# Test 3 : tester la direction
robot = Robot(RectangularRoom(5, 8), 1.0)
d = robot.getRobotDirection()
assert 0 <= d < 360

In [ ]:
# Test 4 : modifier la position
room = RectangularRoom(5, 8)
robot = Robot(room, 1.0)
pos = robot.getRobotPosition()
assert 0 <= pos.getX() < 5
assert 0 <= pos.getY() < 8

for _ in range(100):
    x, y = 10 * random.random(), 10 * random.random()
    pos = Position(x, y)
    if room.isPositionInRoom(pos):
        robot.setRobotPosition(pos)
        p1 = robot.getRobotPosition()
        assert x == p1.getX()
        assert y == p1.getY()

In [ ]:
# Test 5 : modifier la direction
room = RectangularRoom(5, 8)
robot = Robot(room, 1.0)
d = robot.getRobotDirection()
assert 0 <= d < 360

for _ in range(100):
    d = 360 * random.random()
    robot.setRobotDirection(d)
    d1 = robot.getRobotDirection()
    assert d == d1

In [ ]:
# Test 6 : la classe Robot ne doit pas définir la méthode updatePositionAndClean
robot = Robot(RectangularRoom(5, 8), 1.0)
try:
    robot.updatePositionAndClean()
    assert False
except NotImplementedError:
    assert True

## La classe `StandardRobot`
Chaque robot doit également recevoir des instructions pour se déplacer dans la pièce. C'est le but de la méthode `updatePositionAndClean`.

Il est possible de n'utiliser qu'une unique classe et plusieurs méthodes qui lui serait associer. Toutefois, afin de pouvoir envisager différentes stratégies pour les déplacements, il sera possible d'utiliser différentes classes avec la même interface. Ces classes auront donc différentes implémentation pour la méthode `updatePositionAndClean` mais restent, en grande partie, similaires à la classe `Robot` originale. Il est donc intéressant d'utiliser la notion d'héritage pour réduire la duplication du code.

La classe `Robot` a déjà été implémentée et la classe `StandardRobot` en est donc une sous classe.

Il faut complèter la méthode `updatePositionAndClean` de la classe `StandardRobot` pour simuler le mouvement du robot après un simple laps de temps.

### Remarque :
La classe `Position` possède la méthode `getNewPosition` qui  pourra vous être utile.

In [ ]:
class StandardRobot(Robot):
    """
    A StandardRobot is a Robot with the standard movement strategy.

    At each time-step, a StandardRobot attempts to move in its current
    direction; when it would hit a wall, it *instead* chooses a new direction
    randomly.
    """
    def updatePositionAndClean(self):
        """
        Simulate the passage of a single time-step.

        Move the robot to a new position and mark the tile it is on as having
        been cleaned.
        """
        raise NotImplementedError

In [ ]:
# Test 1 : créé un robot
robot = StandardRobot(RectangularRoom(1, 2), 1.0)

In [ ]:
# Test 2 : setRobotPosition
room = RectangularRoom(5, 8)
robot = StandardRobot(room , 1.0)
pos = robot.getRobotPosition()
assert 0 <= pos.getX() < 5
assert 0 <= pos.getY() < 8

for _ in range(100):
    x, y = 10 * random.random(), 10 * random.random()
    pos = Position(x, y)
    if room.isPositionInRoom(pos):
        robot.setRobotPosition(pos)
        p1 = robot.getRobotPosition()
        assert x == p1.getX()
        assert y == p1.getY()

In [ ]:
# Test 3 : setRobotDirection
room = RectangularRoom(5, 8)
robot = StandardRobot(room , 1.0)
d = robot.getRobotDirection()
assert 0 <= d < 360

for _ in range(100):
    d = 360 * random.random()
    robot.setRobotDirection(d)
    d1 = robot.getRobotDirection()
    assert d == d1

In [ ]:
# Test 4 : updatePositionAndClean()
room = RectangularRoom(5, 8)
robot = StandardRobot(room , 1.0)
robot.setRobotPosition(Position(1.5, 2.5))
robot.setRobotDirection(90)
robot.updatePositionAndClean()
for _ in range(20):
    robot.updatePositionAndClean()

In [ ]:
# Test 5 : updatePositionAndClean() and clean
room = RectangularRoom(10, 7)
robot = StandardRobot(room , 0.77)
carreaux = set()
pos = robot.getRobotPosition()
x, y = int(pos.getX()), int(pos.getY())
assert room.isTileCleaned(x, y) == True
carreaux.add((x, y))
assert room.getNumCleanedTiles() == 1

for _ in range(20):
    robot.updatePositionAndClean()
    pos = robot.getRobotPosition()
    x, y = int(pos.getX()), int(pos.getY())
    assert room.isTileCleaned(x, y) == True
    carreaux.add((x, y))
assert room.getNumCleanedTiles() == len(carreaux)

## Simulation
Il faut implémenter un code qui lance une simulation pour le robot.

On rappelle que, pour chaque essai, l'objectif est de déterminer le nombre de pas sont nécessaires, en moyenne, pour qu'une certaine proportion de la pièce soit nettoyée.

Il faut implémenter la fonction `runSimulation()` en respectant les spécifications des docstrings.

Pour l'instant, il faut passer le paramètre `StandardRobot` pour l'argument `robot_type`.

```python
avg = runSimulation(10, 1.0, 15, 20, 0.8, 30, StandardRobot)
```

Cependant, dans le corps de la fonction `runSimulation`, il faut utiliser `robot_type(...)` et, non pas, `StandardRobot(...)` à chaque fois que l'on souhaite créer une instance de robot. Cela permettra d'adapter la simulation aux différentes implémentations de robots.

Voici quelques références approximatives pour les temps de nettoyage (ces temps sont donnés pour des robots de vitesse `1.0`).
* Un robot prend environ $150$ unités de temps pour nettoyer une pièce $5 \times 5$.
* Un robot prend environ $190$ unités de temps pour nettoyer $75\%$ d'une pièce $10 \times 10$.
* Un robot prend environ $310$ unités de temps pour nettoyer $90\%$ d'une pièce $10 \times 10$.
* Un robot prend environ $3322$ unités de temps pour nettoyer une pièce $20 \times 20$.
* Trois robots prend environ $1105$ unités de temps pour nettoyer une pièce $20 \times 20$.

In [ ]:
def runSimulation(num_robots, speed, width, height, min_coverage, num_trials, robot_type):
    """
    Runs NUM_TRIALS trials of the simulation and returns the mean number of
    time-steps needed to clean the fraction MIN_COVERAGE of the room.

    The simulation is run with NUM_ROBOTS robots of type ROBOT_TYPE, each with
    speed SPEED, in a room of dimensions WIDTH x HEIGHT.

    num_robots: an int (num_robots > 0)
    speed: a float (speed > 0)
    width: an int (width > 0)
    height: an int (height > 0)
    min_coverage: a float (0 <= min_coverage <= 1.0)
    num_trials: an int (num_trials > 0)
    robot_type: class of robot to be instantiated (e.g. StandardRobot or
                RandomWalkRobot)
    """
    raise NotImplementedError

## Visualisation
Voici une fonctionnalité qui pourra être utile pour le débogage.  

Un [module](Modules/robot_visualize.py) est fourni pour visualiser des animations de vos robots, lorsqu'ils nettoient la pièce. Ces animations peuvent aider à mieux comprendre le fonctionnement du robot et détecter certains problèmes d'implémentations lorsque les choses ne se passent pas comme prévu.

### Utilisation
* Il faudra donc tout d'abord importer le module :

```python
import robot_visualize
```

* Dans la simulation (dans le code de la fonction `runSimulation`), au début d'un essai, il faut insérer la ligne de code suivante pour lancer une animation :

```python
anim = robot_visualize.RobotVisualization(num_robots, width, height)
```

* Puis, **pour chaque unité de temps**, avant que le robot ne se déplace, insérer la ligne de code suivante pour dessiner une nouvelle image pour l'animation :
```python
anim.update(room, robots)
```  
avec `room` qui est un objet `RectangularRoom` et `robots` une liste d'objets `Robot` pour représenter l'état actuel état de la pièce et les robots dans la pièce.

* Enfin lorsque l'essai est terminé, appeler la méthode suivante :

```python
anim.done()
```

### Remarques
* Le code permettant la visualisation ralentit la simulation. Il faudra donc désactiver les codes d'animation (en commentant les lignes de codes utilisées pour l'animation) si l'on souhaite effectuer des simulations avec un grand nombre d'essais. 
* Pour déboguer la simulation, il est possible de ralentir l'animation. C'est possible en modifiant l'appel à `RobotVisualization`, de la manière suivante :
```python
anim = robot_visualize.RobotVisualization(num_robots, width, height, delay)
```
Le paramètre `delay` défini le nombre de secondes de pause entre les images de l'animation. La valeur, par défaut, est `0.2` (ce qui correspond à 5 images par secondes). Il est possible d'augmenter cette valeur pour rendre l'animation plus lente ou diminuer (par exemple, `0.01`) pour observer différents robots nettoyer la pièce à un dévit plus rapide.

### Exemples
Voici quelques exemples d'utilisation de la simulation et l'animation correspondante.  
L'animation s'exécute lorsqu'un essai est en cours d'exécution dans la fonction `runSimulation`, aussi, il suffit d'appeler la fonction `runSimulation` avec le paramètre `num_trials = 1` : dans ce cas l'objectif n'est pas d'effectuer la simulation sur un grand nombre d'essais pour en déduire une moyenne, mais simplement d'abserver le comportement du robot pour vois s'il est en adéquation avec ce qui est attendu.

* 1 robot, pour une pièce de $5 \times 5$, avec une vitesse de $1.0$ pour nettoyer $100 \%$  de la pièce.

```python
runSimulation(1, 1.0, 5, 5, 1.0, 1, StandardRobot)
```

<div style="text-align: center">
<img src="Images/anim1.gif" alt="Animation">
</div>

* 3 robots, pour une pièce de $15 \times 10$, avec une vitesse de $1.0$ pour nettoyer $80 \%$  de la pièce.

```python
runSimulation(3, 1.0, 15, 10, 0.8, 1, StandardRobot)
```

<div style="text-align: center">
<img src="Images/anim2.gif" alt="Animation">
</div>

* 5 robots, pour une pièce de $15 \times 20$, avec une vitesse de $1.0$ pour nettoyer $75 \%$  de la pièce.

```python
runSimulation(5, 1.0, 15, 20, 0.75, 1, StandardRobot)
```

<div style="text-align: center">
<img src="Images/anim3.gif" alt="Animation">
</div>

## La classe `RandomWalkRobot`
La société souhaitant commercialiser les robots veut tester un nouveau type de robot. Le prinicpe de ce nouveau robot est de changer de direction aléatoirement, **à chaque unité de temps** (et non pas, uniquement, lorsque le robot rencontre un mur).  

Il faut donc mettre en place une simulation, pour évaluer, cette nouvelle approche et les effets sur les temps nécessaires au nettoyage de la pièce.

Il faut complèter la méthode `updatePositionAndClean` de la classe `RandomWalkRobot`, qui hérite également de la classe `Robot` pour simuler le mouvement du robot après un simple laps de temps.  
`RandomWalkRobot` possède donc la même interface que la classe `StandardRobot`.

Il faut donc tester la nouvelle classe. On pourra également s'aider de la fonction `runSimulation` pour vérifier que le robot se comporte de la manière attendue.

Exemple :

* 1 robot, pour une pièce de $5 \times 5$, avec une vitesse de $1.0$ pour nettoyer $100 \%$  de la pièce.

```python
runSimulation(1, 1.0, 5, 5, 1.0, 1, RandomWalkRobot)
```

<div style="text-align: center">
<img src="Images/anim4.gif" alt="Animation">
</div>

* 3 robots, pour une pièce de $15 \times 10$, avec une vitesse de $1.0$ pour nettoyer $80 \%$  de la pièce.

```python
runSimulation(3, 1.0, 15, 10, 0.8, 1, RandomWalkRobot)
```

<div style="text-align: center">
<img src="Images/anim5.gif" alt="Animation">
</div>

* 5 robots, pour une pièce de $15 \times 20$, avec une vitesse de $1.0$ pour nettoyer $75 \%$  de la pièce.

```python
runSimulation(5, 1.0, 15, 20, 0.75, 1, RandomWalkRobot)
```

<div style="text-align: center">
<img src="Images/anim6.gif" alt="Animation">
</div>

In [ ]:
class RandomWalkRobot(Robot):
    """
    A RandomWalkRobot is a robot with the "random walk" movement strategy: it
    chooses a new direction at random at the end of each time-step.
    """
    def updatePositionAndClean(self):
        """
        Simulate the passage of a single time-step.

        Move the robot to a new position and mark the tile it is on as having
        been cleaned.
        """
        raise NotImplementedError

In [ ]:
# Test 1 : créé un robot
robot = RandomWalkRobot(RectangularRoom(1, 2), 1.0)

In [ ]:
# Test 2 : setRobotPosition
room = RectangularRoom(5, 8)
robot = RandomWalkRobot(room , 1.0)
pos = robot.getRobotPosition()
assert 0 <= pos.getX() < 5
assert 0 <= pos.getY() < 8

for _ in range(100):
    x, y = 10 * random.random(), 10 * random.random()
    pos = Position(x, y)
    if room.isPositionInRoom(pos):
        robot.setRobotPosition(pos)
        p1 = robot.getRobotPosition()
        assert x == p1.getX()
        assert y == p1.getY()

In [ ]:
# Test 3 : setRobotDirection
room = RectangularRoom(5, 8)
robot = RandomWalkRobot(room , 1.0)
d = robot.getRobotDirection()
assert 0 <= d < 360

for _ in range(100):
    d = 360 * random.random()
    robot.setRobotDirection(d)
    d1 = robot.getRobotDirection()
    assert d == d1

In [ ]:
# Test 4 : updatePositionAndClean()
room = RectangularRoom(5, 8)
robot = RandomWalkRobot(room , 1.0)
robot.setRobotPosition(Position(1.5, 2.5))
robot.setRobotDirection(90)
robot.updatePositionAndClean()
for _ in range(20):
    robot.updatePositionAndClean()

In [ ]:
# Test 5 : updatePositionAndClean() and clean
room = RectangularRoom(10, 7)
robot = RandomWalkRobot(room , 0.77)
carreaux = set()
pos = robot.getRobotPosition()
x, y = int(pos.getX()), int(pos.getY())
assert room.isTileCleaned(x, y) == True
carreaux.add((x, y))
assert room.getNumCleanedTiles() == 1

for _ in range(20):
    robot.updatePositionAndClean()
    pos = robot.getRobotPosition()
    x, y = int(pos.getX()), int(pos.getY())
    assert room.isTileCleaned(x, y) == True
    carreaux.add((x, y))
assert room.getNumCleanedTiles() == len(carreaux)

On peut ainsi mettre en place différentes stratégies pour que les robots puissent nettoyer une pièce et comparer leur efficacité.